### Talk Time Data Prepare

In [47]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from nltk import ngrams
from functools import reduce
from pathlib import Path
import shutil

In [48]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [49]:
# language pair
lang_folder = "French"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 200  # 28 native word end index

# word usage
min_word_use_num = 1
max_word_use_num = 6

# youtube
sample_num = 6  # 6
time_shift = 0.6

In [50]:
def word_in_wordgroup_simple(source_word_list, df_target, target_column, word_sample_num, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column row result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,"search_string",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [51]:
def word_group_youtube(df, search_list, target_column, sample_num):
    '''
    word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)\n
    df_youtube_sentence is dataframe and "sentence" is its column for external searching_list
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].sample(sample_num)
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].head(sample_num)
        #df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]  # sentence length part
        #df_result.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        #df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [52]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")\n
    df_search_result is dataframe and "search_string", "start_time", "end_time", "sentence", "video_id" are its columns
    '''
    df.reset_index(drop=True, inplace=True)
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [53]:
def word_count_result(df, column_list, set_condition=False): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        if set_condition:
            var_list = set(df[f"{i}"].dropna().tolist())
        else:
            var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    df_word_count.sort_values("word_count", ascending=False, inplace=True)
    df_word_count.reset_index(inplace=True, drop=True)
    
    return  df_word_count

In [54]:
def word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max):
    '''
    word_usage_result(word_list, df_threegram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value,opt_value]) 
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    df_result.sort_values(by="frequency", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [55]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Data/1-Talk Time Data Prepare/{lang_folder.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [56]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,de,16588988
1,je,16386475
2,pas,11547876
3,le,10592792
4,la,9939090
...,...,...
317922,puisetil,5
317923,shouzheng,5
317924,steinam,5
317925,cochezle,5


In [57]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [58]:
df_word_select

,word,frequency
0,de,16588988
1,je,16386475
2,pas,11547876
3,le,10592792
4,la,9939090
...,...,...
195,nos,332905
196,cela,330730
197,nom,329252
198,prendre,327639


In [59]:
df_word_select.to_excel(f"{lang_folder.capitalize()}_200_Word.xlsx", index=False)

In [60]:
word_list = df_word_select["word"].to_list()
len(word_list)

200

In [61]:
df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Two_Gram_And_Sentence.csv")  # ext. sentence and ngram
df_twogram_sent.rename(columns={"two_gram":"twogram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
df_twogram_sent

,twogram,frequency
0,bien sûr,132006
1,je sais,130876
2,ça va,120098
3,très bien,107245
4,cest vrai,98435
...,...,...
468839,salut chicago,2
468840,salut chickamaw,2
468841,salut ching,2
468842,salut chienpo,2


In [62]:
# Twogram Func
d_list2  = df_twogram_sent.iloc[:,0].values.tolist()

resultlist2 = []
manager = multiprocessing.Manager()
resultlist2 = manager.list()

def word_in_wordgroup(d_list2):
    mergelist2 = []
    try:
        word = d_list2.split()
    except:
        word = []
        #pass  disabled for non split value
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist2.append(word[j])
            if len(mergelist2) == len(word):
                    resultlist2.append(d_list2)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list2) # string_word liste

result_list2 = list(resultlist2)
df_result2 = pd.DataFrame(result_list2, columns=[0])  # add columns parameter for empty result
df_result2 = df_result2.rename(columns = {0: "twogram"})
df_merge2 = pd.merge(df_result2, df_twogram_sent, how="left", on="twogram")
df_merge_result2 = df_merge2.sort_values(by="frequency", ascending=False)
df_merge_result2.drop_duplicates(inplace=True)
df_merge_result2.reset_index(drop=True, inplace=True)
df_twogram_select = df_merge_result2
df_twogram_select

,twogram,frequency
0,bien sûr,132006
1,je sais,130876
2,ça va,120098
3,très bien,107245
4,cest vrai,98435
...,...,...
5239,comme mon,2
5240,oh deux,2
5241,homme bien,2
5242,pas son,2


In [63]:
df_threegram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Three_Gram_And_Sentence.csv")  # ext. sentence and ngram
df_threegram_sent.rename(columns={"three_gram":"threegram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
df_threegram_sent

,threegram,frequency
0,oh mon dieu,76182
1,je suis désolé,60108
2,tout va bien,52437
3,questce que cest,49666
4,je sais pas,47754
...,...,...
910305,un petit retard,2
910306,bonsoir m romano,2
910307,un petit rein,2
910308,un petit regard,2


In [64]:
# Threegram Func
d_list3  = df_threegram_sent.iloc[:,0].values.tolist()

resultlist3 = []
manager = multiprocessing.Manager()
resultlist3 = manager.list()

def word_in_wordgroup(d_list3):
    mergelist3 = []
    try:
        word = d_list3.split()
    except:
        word = []
        #pass  disabled for non split value
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist3.append(word[j])
            if len(mergelist3) == len(word):
                    resultlist3.append(d_list3)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list3) # string_word liste

result_list3 = list(resultlist3)
df_result3 = pd.DataFrame(result_list3, columns=[0])  # add columns parameter for empty result
df_result3 = df_result3.rename(columns = {0: "threegram"})
df_merge3 = pd.merge(df_result3, df_threegram_sent, how="left", on="threegram")
df_merge_result3 = df_merge3.sort_values(by="frequency", ascending=False)
df_merge_result3.drop_duplicates(inplace=True)
df_merge_result3.reset_index(drop=True, inplace=True)
df_threegram_select = df_merge_result3
df_threegram_select

,threegram,frequency
0,oh mon dieu,76182
1,je suis désolé,60108
2,tout va bien,52437
3,questce que cest,49666
4,je sais pas,47754
...,...,...
19957,les pour toujours,2
19958,oui ma maison,2
19959,quil vous faut,2
19960,pour leur dire,2


In [65]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.000,00:00:00.200,perrosguirec bretagne,BU7DYGMSUA4
1,00:00:30.972,00:00:33.609,located in the regional natural park of armorica,4nmWfnmVgo4
2,00:00:34.263,00:00:36.866,the huelgoat forest,4nmWfnmVgo4
3,00:00:37.315,00:00:39.315,has always inspired popular stories,4nmWfnmVgo4
4,00:00:40.132,00:00:43.640,places with mysterious names sometimes refer t...,4nmWfnmVgo4
...,...,...,...,...
477511,01:17:00.530,01:17:07.970,être remettre le lien si vous avez des questio...,gloQzChn1lk
477512,01:17:07.970,01:17:13.640,la crise sanitaire ou quils concernent autre c...,gloQzChn1lk
477513,01:17:13.640,01:17:20.960,contacter et en tout cas merci beaucoup et bon...,gloQzChn1lk
477514,01:17:20.960,01:17:27.140,si catherine merci à tout le monde à revoir me...,gloQzChn1lk


In [66]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [67]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,0.000,0.200,perrosguirec bretagne,BU7DYGMSUA4
1,30.972,33.609,located in the regional natural park of armorica,4nmWfnmVgo4
2,34.263,36.866,the huelgoat forest,4nmWfnmVgo4
3,37.315,39.315,has always inspired popular stories,4nmWfnmVgo4
4,40.132,43.640,places with mysterious names sometimes refer t...,4nmWfnmVgo4
...,...,...,...,...
477511,4620.530,4627.970,être remettre le lien si vous avez des questio...,gloQzChn1lk
477512,4627.970,4633.640,la crise sanitaire ou quils concernent autre c...,gloQzChn1lk
477513,4633.640,4640.960,contacter et en tout cas merci beaucoup et bon...,gloQzChn1lk
477514,4640.960,4647.140,si catherine merci à tout le monde à revoir me...,gloQzChn1lk


In [68]:
df_sentence_link = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Data/{lang_folder.capitalize()}/{lang_folder.capitalize()}_200_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample_Selected.xlsx")
df_sentence_link = df_sentence_link.iloc[:,[0,1,2,3,4,5]]
df_sentence_link

,search_string,start_time,end_time,sentence,video_id,video_url
0,comment ça on vous veut du mal pas du tout on ...,261,266,comment ça on vous veut du mal pas du tout on ...,DFejViks2uY,https://www.youtube.com/watch?v=DFejViks2uY&t=...
1,quand même quoi donc parce que voilà cest fait...,157,163,quand même quoi donc parce que voilà cest fait...,0_sv895JPlc,https://www.youtube.com/watch?v=0_sv895JPlc&t=...
2,nai pas bien fait parce que cest pas bien de f...,591,596,nai pas bien fait parce que cest pas bien de f...,9RDA9RZLTSc,https://www.youtube.com/watch?v=9RDA9RZLTSc&t=...
3,dit il sest passé quelque chose il sest passé ...,257,261,dit il sest passé quelque chose il sest passé ...,hecMN45bbK4,https://www.youtube.com/watch?v=hecMN45bbK4&t=...
4,nom mais je pense que jétais en mieux et peut ...,135,144,nom mais je pense que jétais en mieux et peut ...,c1zhja4N8hw,https://www.youtube.com/watch?v=c1zhja4N8hw&t=...
5,là pourquoi je suis là et vous pourquoi vous ê...,672,676,ce que je fous là pourquoi je suis là et vous ...,jpwUNmNViuk,https://www.youtube.com/watch?v=jpwUNmNViuk&t=...
6,comme ça mais je pense que cest une bonne chos...,566,571,démonter comme ça mais je pense que cest une b...,7rV0nsyIiUg,https://www.youtube.com/watch?v=7rV0nsyIiUg&t=...
7,as été bien daccord avec moi elle était pas ét...,175,186,as été bien daccord avec moi elle était pas ét...,l7kuaPv5UvA,https://www.youtube.com/watch?v=l7kuaPv5UvA&t=...
8,de dire dans le passé ce qui était bien et ce ...,34,37,de dire dans le passé ce qui était bien et ce ...,naM2K14xXqI,https://www.youtube.com/watch?v=naM2K14xXqI&t=34s
9,elle est bien elle est bien mais elle est très...,287,293,elle est bien elle est bien mais elle est très...,OS_T0FMHoBY,https://www.youtube.com/watch?v=OS_T0FMHoBY&t=...


In [69]:
twogram_set = set()
threegram_set = set()
for sent in df_sentence_link["search_string"]:
    twogram_zip = ngrams(sent.split(), 2)
    for two in twogram_zip:
        twogram = " ".join(two)
        twogram_set.add(f"{twogram}") 
    threegram_zip = ngrams(sent.split(), 3)
    for three in threegram_zip:
        threegram = " ".join(three)
        threegram_set.add(f"{threegram}")

In [70]:
len(twogram_set)

147

In [71]:
len(threegram_set)

156

In [72]:
twogram_list = list(twogram_set)
#twogram_list

In [73]:
threegram_list = list(threegram_set)
#threegram_list

In [74]:
df_twogram_for_link = df_twogram_select[df_twogram_select["twogram"].isin(twogram_list)]
df_twogram_for_link.reset_index(drop=True, inplace=True)
df_twogram_for_link

,twogram,frequency
0,comment ça,44736
1,et vous,18817
2,mais non,18267
3,de quoi,16719
4,comme ça,12528
...,...,...
80,voilà cest,2
81,était mal,2
82,moi elle,2
83,est très,2


In [75]:
df_twogram_selected_for_link = word_usage_result(word_list, df_twogram_for_link, "twogram", "frequency", min_word_use_num, max_word_use_num)
df_twogram_selected_for_link 

,twogram,frequency
0,comment ça,44736
1,et vous,18817
2,mais non,18267
3,de quoi,16719
4,comme ça,12528
...,...,...
74,que voilà,4
75,était mal,2
76,moi elle,2
77,est très,2


In [76]:
df_threegram_for_link = df_threegram_select[df_threegram_select["threegram"].isin(threegram_list)]
df_threegram_for_link.reset_index(drop=True, inplace=True)
df_threegram_for_link

,threegram,frequency
0,pas du tout,29926
1,je suis là,15121
2,vous êtes là,3591
3,cest pas bien,1223
4,je pense que,881
5,elle est bien,699
6,voilà cest fait,339
7,comment ça on,175
8,dans le passé,115
9,cest ce que,112


In [77]:
df_threegram_selected_for_link = word_usage_result(word_list, df_threegram_for_link, "threegram", "frequency", min_word_use_num, max_word_use_num)
df_threegram_selected_for_link 

,threegram,frequency
0,pas du tout,29926
1,je suis là,15121
2,vous êtes là,3591
3,cest pas bien,1223
4,je pense que,881
5,elle est bien,699
6,voilà cest fait,339
7,comment ça on,175
8,dans le passé,115
9,cest ce que,112


In [78]:
df_twogram_search_result = word_group_youtube(df_youtube_sentence, df_twogram_selected_for_link["twogram"], "sentence", sample_num)
df_twogram_search_result

,search_string,start_time,end_time,sentence,video_id
0,comment ça,730.630,740.050,noël ma absolument oui bernois comment ça va a...,KTujQiqqtcs
1,et vous,1372.640,1374.320,chrétiens et vous savez si vous êtes un vrai,gr_2UsdqPWE
2,mais non,193.960,197.360,jespère que je ne lui ai pas fait trop mal mai...,PfVqyogksgs
3,de quoi,200.080,202.440,de quoi ont besoin les plantes pour être heure...,DhEqSOjwYq0
4,comme ça,3988.815,3991.418,très bien comme ça parce que,H4NOeTTLAic
...,...,...,...,...,...
74,que voilà,1923.630,1929.720,ça qui apparaîtra je crois que voilà vraiment ...,hlQNYZMJz2c
75,était mal,1199.790,1204.770,était mal positionné eh bien je vais en tirer ...,vknHaRqCPvk
76,moi elle,519.367,521.367,cette notion de enchantement pour moi elle est...,vQWSJoODS28
77,est très,53.980,58.949,concernant ce master de psycho qui est très de...,1bSHE3sH0mE


In [79]:
df_word_group_time_loc_twogram_result = word_group_time_loc(df_twogram_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_twogram_result

,search_string,start_time,end_time,sentence,video_id
0,comment ça,733.573750,734.751250,noël ma absolument oui bernois comment ça va a...,KTujQiqqtcs
1,et vous,1372.983636,1373.327273,chrétiens et vous savez si vous êtes un vrai,gr_2UsdqPWE
2,mais non,196.300984,196.858361,jespère que je ne lui ai pas fait trop mal mai...,PfVqyogksgs
3,de quoi,200.080000,200.457600,de quoi ont besoin les plantes pour être heure...,DhEqSOjwYq0
4,comme ça,3989.651679,3990.581321,très bien comme ça parce que,H4NOeTTLAic
...,...,...,...,...,...
74,que voilà,1925.197723,1925.860990,ça qui apparaîtra je crois que voilà vraiment ...,hlQNYZMJz2c
75,était mal,1199.790000,1200.349551,était mal positionné eh bien je vais en tirer ...,vknHaRqCPvk
76,moi elle,520.431516,520.754097,cette notion de enchantement pour moi elle est...,vQWSJoODS28
77,est très,57.108630,58.028815,concernant ce master de psycho qui est très de...,1bSHE3sH0mE


In [80]:
df_word_group_time_loc_twogram_result.start_time = df_word_group_time_loc_twogram_result.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_twogram_result.end_time = df_word_group_time_loc_twogram_result.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_twogram_result

,search_string,start_time,end_time,sentence,video_id
0,comment ça,732.973750,735.351250,noël ma absolument oui bernois comment ça va a...,KTujQiqqtcs
1,et vous,1372.383636,1373.927273,chrétiens et vous savez si vous êtes un vrai,gr_2UsdqPWE
2,mais non,195.700984,197.458361,jespère que je ne lui ai pas fait trop mal mai...,PfVqyogksgs
3,de quoi,199.480000,201.057600,de quoi ont besoin les plantes pour être heure...,DhEqSOjwYq0
4,comme ça,3989.051679,3991.181321,très bien comme ça parce que,H4NOeTTLAic
...,...,...,...,...,...
74,que voilà,1924.597723,1926.460990,ça qui apparaîtra je crois que voilà vraiment ...,hlQNYZMJz2c
75,était mal,1199.190000,1200.949551,était mal positionné eh bien je vais en tirer ...,vknHaRqCPvk
76,moi elle,519.831516,521.354097,cette notion de enchantement pour moi elle est...,vQWSJoODS28
77,est très,56.508630,58.628815,concernant ce master de psycho qui est très de...,1bSHE3sH0mE


In [81]:
df_word_group_time_loc_twogram_result.start_time = df_word_group_time_loc_twogram_result.start_time.apply(lambda x: round(x))
df_word_group_time_loc_twogram_result.end_time = df_word_group_time_loc_twogram_result.end_time.apply(lambda x: round(x))
df_word_group_time_loc_twogram_result 

,search_string,start_time,end_time,sentence,video_id
0,comment ça,733,735,noël ma absolument oui bernois comment ça va a...,KTujQiqqtcs
1,et vous,1372,1374,chrétiens et vous savez si vous êtes un vrai,gr_2UsdqPWE
2,mais non,196,197,jespère que je ne lui ai pas fait trop mal mai...,PfVqyogksgs
3,de quoi,199,201,de quoi ont besoin les plantes pour être heure...,DhEqSOjwYq0
4,comme ça,3989,3991,très bien comme ça parce que,H4NOeTTLAic
...,...,...,...,...,...
74,que voilà,1925,1926,ça qui apparaîtra je crois que voilà vraiment ...,hlQNYZMJz2c
75,était mal,1199,1201,était mal positionné eh bien je vais en tirer ...,vknHaRqCPvk
76,moi elle,520,521,cette notion de enchantement pour moi elle est...,vQWSJoODS28
77,est très,57,59,concernant ce master de psycho qui est très de...,1bSHE3sH0mE


In [82]:
df_word_group_time_loc_twogram_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_twogram_result['video_id'].map(str)+"&t="+df_word_group_time_loc_twogram_result['start_time'].map(str)+"s"
df_word_group_time_loc_twogram_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,comment ça,733,735,noël ma absolument oui bernois comment ça va a...,KTujQiqqtcs,https://www.youtube.com/watch?v=KTujQiqqtcs&t=...
1,et vous,1372,1374,chrétiens et vous savez si vous êtes un vrai,gr_2UsdqPWE,https://www.youtube.com/watch?v=gr_2UsdqPWE&t=...
2,mais non,196,197,jespère que je ne lui ai pas fait trop mal mai...,PfVqyogksgs,https://www.youtube.com/watch?v=PfVqyogksgs&t=...
3,de quoi,199,201,de quoi ont besoin les plantes pour être heure...,DhEqSOjwYq0,https://www.youtube.com/watch?v=DhEqSOjwYq0&t=...
4,comme ça,3989,3991,très bien comme ça parce que,H4NOeTTLAic,https://www.youtube.com/watch?v=H4NOeTTLAic&t=...
...,...,...,...,...,...,...
74,que voilà,1925,1926,ça qui apparaîtra je crois que voilà vraiment ...,hlQNYZMJz2c,https://www.youtube.com/watch?v=hlQNYZMJz2c&t=...
75,était mal,1199,1201,était mal positionné eh bien je vais en tirer ...,vknHaRqCPvk,https://www.youtube.com/watch?v=vknHaRqCPvk&t=...
76,moi elle,520,521,cette notion de enchantement pour moi elle est...,vQWSJoODS28,https://www.youtube.com/watch?v=vQWSJoODS28&t=...
77,est très,57,59,concernant ce master de psycho qui est très de...,1bSHE3sH0mE,https://www.youtube.com/watch?v=1bSHE3sH0mE&t=57s


In [83]:
df_word_group_time_loc_twogram_result.to_excel(f"{lang_folder.capitalize()}_Twogram_With_{word_end}_Word_{sample_num}_Youtube_{time_shift}s_Timeshift_For_Talk_Time_Result.xlsx", index=False) 

In [84]:
df_threegram_search_result = word_group_youtube(df_youtube_sentence, df_threegram_selected_for_link["threegram"], "sentence", sample_num)
df_threegram_search_result

,search_string,start_time,end_time,sentence,video_id
0,pas du tout,5907.624,5912.934,pas du tout été traités le bilan dramatique en...,LSZIjPaZTb4
1,je suis là,29.080,32.320,je suis là pour faire une mise à leau,3QblW0a8IxI
2,vous êtes là,671.260,675.610,ce que je fous là pourquoi je suis là et vous ...,jpwUNmNViuk
3,cest pas bien,905.160,905.968,cest pas bien grave,Qe51q3_WMWg
4,je pense que,2232.039,2233.974,et je pense que tant quon a,seWzxrNRuZc
5,elle est bien,85.140,89.700,à deux places ou plus que lon peut facilement ...,MyWOr94A0b4
6,voilà cest fait,182.740,187.930,juste à cliquer sur publish et publish to sele...,prldVLvuAJk
7,comment ça on,261.080,266.640,comment ça on vous veut du mal pas du tout on ...,DFejViks2uY
8,dans le passé,358.720,363.532,jen ai beaucoup ris mais ils possèdent tous de...,yTcxoLGVzG8
9,cest ce que,78.675,82.110,et bien cest ce que nous allons découvrir ou r...,WkEfYhisILE


In [85]:
df_word_group_time_loc_threegram_result = word_group_time_loc(df_threegram_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_threegram_result

,search_string,start_time,end_time,sentence,video_id
0,pas du tout,5907.624000,5908.267636,pas du tout été traités le bilan dramatique en...,LSZIjPaZTb4
1,je suis là,29.080000,30.043243,je suis là pour faire une mise à leau,3QblW0a8IxI
2,vous êtes là,673.841319,674.510549,ce que je fous là pourquoi je suis là et vous ...,jpwUNmNViuk
3,cest pas bien,905.160000,905.755368,cest pas bien grave,Qe51q3_WMWg
4,je pense que,2232.182333,2233.185667,et je pense que tant quon a,seWzxrNRuZc
5,elle est bien,88.828989,89.597528,à deux places ou plus que lon peut facilement ...,MyWOr94A0b4
6,voilà cest fait,186.851558,187.930000,juste à cliquer sur publish et publish to sele...,prldVLvuAJk
7,comment ça on,261.080000,261.712846,comment ça on vous veut du mal pas du tout on ...,DFejViks2uY
8,dans le passé,362.984293,363.532000,jen ai beaucoup ris mais ils possèdent tous de...,yTcxoLGVzG8
9,cest ce que,79.044923,79.731923,et bien cest ce que nous allons découvrir ou r...,WkEfYhisILE


In [86]:
df_word_group_time_loc_threegram_result.start_time = df_word_group_time_loc_threegram_result.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_threegram_result.end_time = df_word_group_time_loc_threegram_result.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_threegram_result

,search_string,start_time,end_time,sentence,video_id
0,pas du tout,5907.024000,5908.867636,pas du tout été traités le bilan dramatique en...,LSZIjPaZTb4
1,je suis là,28.480000,30.643243,je suis là pour faire une mise à leau,3QblW0a8IxI
2,vous êtes là,673.241319,675.110549,ce que je fous là pourquoi je suis là et vous ...,jpwUNmNViuk
3,cest pas bien,904.560000,906.355368,cest pas bien grave,Qe51q3_WMWg
4,je pense que,2231.582333,2233.785667,et je pense que tant quon a,seWzxrNRuZc
5,elle est bien,88.228989,90.197528,à deux places ou plus que lon peut facilement ...,MyWOr94A0b4
6,voilà cest fait,186.251558,188.530000,juste à cliquer sur publish et publish to sele...,prldVLvuAJk
7,comment ça on,260.480000,262.312846,comment ça on vous veut du mal pas du tout on ...,DFejViks2uY
8,dans le passé,362.384293,364.132000,jen ai beaucoup ris mais ils possèdent tous de...,yTcxoLGVzG8
9,cest ce que,78.444923,80.331923,et bien cest ce que nous allons découvrir ou r...,WkEfYhisILE


In [87]:
df_word_group_time_loc_threegram_result.start_time = df_word_group_time_loc_threegram_result.start_time.apply(lambda x: round(x))
df_word_group_time_loc_threegram_result.end_time = df_word_group_time_loc_threegram_result.end_time.apply(lambda x: round(x))
df_word_group_time_loc_threegram_result 

,search_string,start_time,end_time,sentence,video_id
0,pas du tout,5907,5909,pas du tout été traités le bilan dramatique en...,LSZIjPaZTb4
1,je suis là,28,31,je suis là pour faire une mise à leau,3QblW0a8IxI
2,vous êtes là,673,675,ce que je fous là pourquoi je suis là et vous ...,jpwUNmNViuk
3,cest pas bien,905,906,cest pas bien grave,Qe51q3_WMWg
4,je pense que,2232,2234,et je pense que tant quon a,seWzxrNRuZc
5,elle est bien,88,90,à deux places ou plus que lon peut facilement ...,MyWOr94A0b4
6,voilà cest fait,186,189,juste à cliquer sur publish et publish to sele...,prldVLvuAJk
7,comment ça on,260,262,comment ça on vous veut du mal pas du tout on ...,DFejViks2uY
8,dans le passé,362,364,jen ai beaucoup ris mais ils possèdent tous de...,yTcxoLGVzG8
9,cest ce que,78,80,et bien cest ce que nous allons découvrir ou r...,WkEfYhisILE


In [88]:
df_word_group_time_loc_threegram_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_threegram_result['video_id'].map(str)+"&t="+df_word_group_time_loc_threegram_result['start_time'].map(str)+"s"
df_word_group_time_loc_threegram_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,pas du tout,5907,5909,pas du tout été traités le bilan dramatique en...,LSZIjPaZTb4,https://www.youtube.com/watch?v=LSZIjPaZTb4&t=...
1,je suis là,28,31,je suis là pour faire une mise à leau,3QblW0a8IxI,https://www.youtube.com/watch?v=3QblW0a8IxI&t=28s
2,vous êtes là,673,675,ce que je fous là pourquoi je suis là et vous ...,jpwUNmNViuk,https://www.youtube.com/watch?v=jpwUNmNViuk&t=...
3,cest pas bien,905,906,cest pas bien grave,Qe51q3_WMWg,https://www.youtube.com/watch?v=Qe51q3_WMWg&t=...
4,je pense que,2232,2234,et je pense que tant quon a,seWzxrNRuZc,https://www.youtube.com/watch?v=seWzxrNRuZc&t=...
5,elle est bien,88,90,à deux places ou plus que lon peut facilement ...,MyWOr94A0b4,https://www.youtube.com/watch?v=MyWOr94A0b4&t=88s
6,voilà cest fait,186,189,juste à cliquer sur publish et publish to sele...,prldVLvuAJk,https://www.youtube.com/watch?v=prldVLvuAJk&t=...
7,comment ça on,260,262,comment ça on vous veut du mal pas du tout on ...,DFejViks2uY,https://www.youtube.com/watch?v=DFejViks2uY&t=...
8,dans le passé,362,364,jen ai beaucoup ris mais ils possèdent tous de...,yTcxoLGVzG8,https://www.youtube.com/watch?v=yTcxoLGVzG8&t=...
9,cest ce que,78,80,et bien cest ce que nous allons découvrir ou r...,WkEfYhisILE,https://www.youtube.com/watch?v=WkEfYhisILE&t=78s


In [89]:
df_word_group_time_loc_threegram_result.to_excel(f"{lang_folder.capitalize()}_Threegram_With_{word_end}_Word_{sample_num}_Youtube_{time_shift}s_Timeshift_For_Talk_Time_Result.xlsx", index=False) 

In [90]:
df_word_search_result = word_group_youtube(df_youtube_sentence, word_list, "sentence", sample_num)
df_word_search_result

,search_string,start_time,end_time,sentence,video_id
0,de,578.640,583.750,vient le moment où nous comprenons que ce que ...,AkoZt6NCQpA
1,je,156.130,159.340,sont à biarritz ce weekend cest cool mais plut...,S8IncqDM3ic
2,pas,206.560,212.020,médecine parce que pour lui la médecine cétait...,LdZZKGXf5G4
3,le,1538.566,1541.098,le règne de dieu est tout proche,N0x70T8aNVU
4,la,144.989,148.930,de consommer par jour posezvous la question et...,qvOl8x9jxjw
...,...,...,...,...,...
195,nos,581.340,586.740,on va en reparler tout à lheure vous allez voi...,Tndgvn3FoZU
196,cela,697.920,700.522,en chantant cela on imagine déjà,p6FqYFOoFvQ
197,nom,868.450,875.530,jécris ton nom sur la mousse des nuages sur le...,PXUG5yjcUh4
198,prendre,135.300,138.360,au bout dun moment le serveur vint prendre sa ...,OiyQXzGpXZw


In [91]:
df_word_group_time_loc_word_result = word_group_time_loc(df_word_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_word_result

,search_string,start_time,end_time,sentence,video_id
0,de,583.332857,583.541429,vient le moment où nous comprenons que ce que ...,AkoZt6NCQpA
1,je,158.257558,158.406860,sont à biarritz ce weekend cest cool mais plut...,S8IncqDM3ic
2,pas,209.149278,209.430722,médecine parce que pour lui la médecine cétait...,LdZZKGXf5G4
3,le,1538.566000,1538.803375,le règne de dieu est tout proche,N0x70T8aNVU
4,la,146.377307,146.556443,de consommer par jour posezvous la question et...,qvOl8x9jxjw
...,...,...,...,...,...
195,nos,586.092000,586.308000,on va en reparler tout à lheure vous allez voi...,Tndgvn3FoZU
196,cela,698.814437,699.302312,en chantant cela on imagine déjà,p6FqYFOoFvQ
197,nom,869.203191,869.579787,jécris ton nom sur la mousse des nuages sur le...,PXUG5yjcUh4
198,prendre,137.226667,137.736667,au bout dun moment le serveur vint prendre sa ...,OiyQXzGpXZw


In [92]:
df_word_group_time_loc_word_result.start_time = df_word_group_time_loc_word_result.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_word_result.end_time = df_word_group_time_loc_word_result.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_word_result

,search_string,start_time,end_time,sentence,video_id
0,de,582.732857,584.141429,vient le moment où nous comprenons que ce que ...,AkoZt6NCQpA
1,je,157.657558,159.006860,sont à biarritz ce weekend cest cool mais plut...,S8IncqDM3ic
2,pas,208.549278,210.030722,médecine parce que pour lui la médecine cétait...,LdZZKGXf5G4
3,le,1537.966000,1539.403375,le règne de dieu est tout proche,N0x70T8aNVU
4,la,145.777307,147.156443,de consommer par jour posezvous la question et...,qvOl8x9jxjw
...,...,...,...,...,...
195,nos,585.492000,586.908000,on va en reparler tout à lheure vous allez voi...,Tndgvn3FoZU
196,cela,698.214437,699.902312,en chantant cela on imagine déjà,p6FqYFOoFvQ
197,nom,868.603191,870.179787,jécris ton nom sur la mousse des nuages sur le...,PXUG5yjcUh4
198,prendre,136.626667,138.336667,au bout dun moment le serveur vint prendre sa ...,OiyQXzGpXZw


In [93]:
df_word_group_time_loc_word_result.start_time = df_word_group_time_loc_word_result.start_time.apply(lambda x: round(x))
df_word_group_time_loc_word_result.end_time = df_word_group_time_loc_word_result.end_time.apply(lambda x: round(x))
df_word_group_time_loc_word_result 

,search_string,start_time,end_time,sentence,video_id
0,de,583,584,vient le moment où nous comprenons que ce que ...,AkoZt6NCQpA
1,je,158,159,sont à biarritz ce weekend cest cool mais plut...,S8IncqDM3ic
2,pas,209,210,médecine parce que pour lui la médecine cétait...,LdZZKGXf5G4
3,le,1538,1539,le règne de dieu est tout proche,N0x70T8aNVU
4,la,146,147,de consommer par jour posezvous la question et...,qvOl8x9jxjw
...,...,...,...,...,...
195,nos,585,587,on va en reparler tout à lheure vous allez voi...,Tndgvn3FoZU
196,cela,698,700,en chantant cela on imagine déjà,p6FqYFOoFvQ
197,nom,869,870,jécris ton nom sur la mousse des nuages sur le...,PXUG5yjcUh4
198,prendre,137,138,au bout dun moment le serveur vint prendre sa ...,OiyQXzGpXZw


In [94]:
df_word_group_time_loc_word_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_word_result['video_id'].map(str)+"&t="+df_word_group_time_loc_word_result['start_time'].map(str)+"s"
df_word_group_time_loc_word_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,de,583,584,vient le moment où nous comprenons que ce que ...,AkoZt6NCQpA,https://www.youtube.com/watch?v=AkoZt6NCQpA&t=...
1,je,158,159,sont à biarritz ce weekend cest cool mais plut...,S8IncqDM3ic,https://www.youtube.com/watch?v=S8IncqDM3ic&t=...
2,pas,209,210,médecine parce que pour lui la médecine cétait...,LdZZKGXf5G4,https://www.youtube.com/watch?v=LdZZKGXf5G4&t=...
3,le,1538,1539,le règne de dieu est tout proche,N0x70T8aNVU,https://www.youtube.com/watch?v=N0x70T8aNVU&t=...
4,la,146,147,de consommer par jour posezvous la question et...,qvOl8x9jxjw,https://www.youtube.com/watch?v=qvOl8x9jxjw&t=...
...,...,...,...,...,...,...
195,nos,585,587,on va en reparler tout à lheure vous allez voi...,Tndgvn3FoZU,https://www.youtube.com/watch?v=Tndgvn3FoZU&t=...
196,cela,698,700,en chantant cela on imagine déjà,p6FqYFOoFvQ,https://www.youtube.com/watch?v=p6FqYFOoFvQ&t=...
197,nom,869,870,jécris ton nom sur la mousse des nuages sur le...,PXUG5yjcUh4,https://www.youtube.com/watch?v=PXUG5yjcUh4&t=...
198,prendre,137,138,au bout dun moment le serveur vint prendre sa ...,OiyQXzGpXZw,https://www.youtube.com/watch?v=OiyQXzGpXZw&t=...


In [95]:
df_word_group_time_loc_word_result.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_{sample_num}_Youtube_{time_shift}s_Timeshift_For_Talk_Time_Result.xlsx", index=False)

#### Copy Move And Delete

In [100]:
output_file = glob.glob(f"{lang_folder.capitalize()}_*_{sample_num}_Youtube_{time_shift}s_Timeshift_For_Talk_Time_Result.xlsx")
output_file

[]

In [97]:
for y in output_file:
    source = y # source directory
    destination = path
    shutil.copy2(source, destination)

In [99]:
for z in output_file:
    try:
        os.remove(z)
    except:
        pass